In [1]:
import pandas

In [4]:
import pandas as pd
import os

df = pd.read_excel('C:/Users/HP/Downloads/EV업데이트 0729 1846.xlsx')

except_list = ['승인불가', '신청포기', '지원취소']

df = df[~df['신청단계'].isin(except_list)]
df = df[df['신청일자'] == '2025-07-29']

df.shape


(74, 70)

In [5]:
df = pd.read_excel('C:/Users/HP/Downloads/EV업데이트 0728 1807.xlsx')

except_list = ['승인불가', '신청포기', '지원취소']

df = df[~df['신청단계'].isin(except_list)]
df = df[df['신청일자'] == '2025-07-28']

df.shape

(121, 70)

In [12]:
df = pd.read_excel('C:/Users/HP/Downloads/OneDrive_2025-07-30/EV업데이트 0714 1800.xls', header=2)

except_list = ['승인불가', '신청포기', '지원취소']

df = df[~df['신청단계'].isin(except_list)]
df = df[df['신청일자'] == '2025-07-14']

df.shape

(130, 70)

In [ ]:
folder_path = 'C:/Users/HP/Downloads'
except_list = ['승인불가', '신청포기', '지원취소']


날짜_개수_리스트 = []

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder_path, file))
        df = df[~df['신청단계'].isin(except_list)]

    elif file.endswith('.xls'):
        df = pd.read_excel(os.path.join(folder_path, file), header=2)
        df = df[~df['신청단계'].isin(except_list)]
        
    # 파일명에서 '0724' 부분을 추출하여 '2025-07-24' 형식으로 변환 후 필터링
    date_str = file.split()[1]  # '0724' 추출
    신청일자 = f"2025-07-{date_str[2:]}"  # '2025-07-24'로 변환
    df = df[df['신청일자'] == 신청일자]
    
    개수 = df.shape[0]
    날짜_개수_리스트.append({'날짜': 신청일자, '개수': 개수})
        

# 리스트를 데이터프레임으로 변환
result_df = pd.DataFrame(날짜_개수_리스트)

# 엑셀로 저장
result_df.to_excel('신청일자별_개수_집계.xlsx', index=False)



In [ ]:
result = pd.read_excel('신청일자별_개수_집계.xlsx')

folder_path = 'C:/Users/HP/Downloads/11'
except_list = ['승인불가', '신청포기', '지원취소']

날짜_개수_리스트 = []

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder_path, file))
        df = df[~df['신청단계'].isin(except_list)]

    elif file.endswith('.xls'):
        df = pd.read_excel(os.path.join(folder_path, file), header=2)
        df = df[~df['신청단계'].isin(except_list)]
        
    # '취소(포기)\n일자'가 기준날짜인 경우
    date_str = file.split()[1]  # 예: '0724' 또는 '0624' 추출
    month = date_str[:2]
    day = date_str[2:]
    기준날짜 = f"2025-{month}-{day}"
    기준날짜 = pd.to_datetime(기준날짜).strftime('%Y-%m-%d')  # 날짜 포맷 보정
    df = df[df['신청일자'] == 기준날짜]
    
    개수 = df.shape[0]
    날짜_개수_리스트.append({'날짜': 기준날짜, '개수': 개수})

# 새로 구한 날짜별 개수 데이터프레임 생성
new_df = pd.DataFrame(날짜_개수_리스트)

# 기존 result와 합치기 (중복 날짜는 모두 남김)
result = pd.concat([result, new_df], ignore_index=True)
result.to_excel('신청일자별_개수_집계.xlsx', index=False)

In [ ]:

folder_path = 'C:/Users/HP/Downloads/11'
except_list = ['승인불가', '신청포기', '지원취소']

날짜_개수_리스트 = []

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder_path, file))
        df = df[~df['신청단계'].isin(except_list)]

    elif file.endswith('.xls'):
        df = pd.read_excel(os.path.join(folder_path, file), header=2)
        df = df[~df['신청단계'].isin(except_list)]
        
    # '취소(포기)\n일자'가 기준날짜인 경우
    date_str = file.split()[1]  # 예: '0724' 또는 '0624' 추출
    month = date_str[:2]
    day = date_str[2:]
    기준날짜 = f"2025-{month}-{day}"
    기준날짜 = pd.to_datetime(기준날짜).strftime('%Y-%m-%d')  # 날짜 포맷 보정
    df = df[df['신청일자'] == 기준날짜]
    
    개수 = df.shape[0]
    날짜_개수_리스트.append({'날짜': 기준날짜, '개수': 개수})

# 새로 구한 날짜별 개수 데이터프레임 생성
new_df = pd.DataFrame(날짜_개수_리스트)

# 기존 result와 합치기 (중복 날짜는 모두 남김)
result = pd.concat([result, new_df], ignore_index=True)
result.to_excel('신청일자별_개수_집계.xlsx', index=False)

In [50]:
result = pd.read_excel('신청일자별_개수_집계.xlsx')
folder_path = 'C:/Users/HP/Downloads/33'
except_list = ['승인불가', '신청포기', '지원취소']

날짜_개수_리스트 = []

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder_path, file))
        df = df[~df['신청단계'].isin(except_list)]

    elif file.endswith('.xls'):
        df = pd.read_excel(os.path.join(folder_path, file), header=2)
        df = df[~df['신청단계'].isin(except_list)]
        
    # '취소(포기)\n일자'가 기준날짜인 경우
    date_str = file.split()[1]  # 예: '0724' 또는 '0624' 추출
    month = date_str[:2]
    day = date_str[2:]
    기준날짜 = f"2025-{month}-{day}"
    기준날짜 = pd.to_datetime(기준날짜).strftime('%Y-%m-%d')  # 날짜 포맷 보정
    df = df[df['신청일자'] == 기준날짜]
    
    개수 = df.shape[0]
    날짜_개수_리스트.append({'날짜': 기준날짜, '개수': 개수})

# 새로 구한 날짜별 개수 데이터프레임 생성
new_df = pd.DataFrame(날짜_개수_리스트)

# 기존 result와 합치기 (중복 날짜는 모두 남김)
result = pd.concat([result, new_df], ignore_index=True)
result.to_excel('신청일자별_개수_집계.xlsx', index=False)

In [53]:
folder_path = 'C:/Users/HP/Downloads/22'
except_list = ['승인불가', '신청포기', '지원취소']

기준날짜_취소_개수_리스트 = []

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder_path, file))
        df = df[df['신청단계'].isin(except_list)]
    elif file.endswith('.xls'):
        df = pd.read_excel(os.path.join(folder_path, file), header=2)
        df = df[df['신청단계'].isin(except_list)]

    # '취소(포기)\n일자'가 기준날짜인 경우
    date_str = file.split()[1]  # 예: '0724' 또는 '0624' 추출
    month = date_str[:2]
    day = date_str[2:]
    기준날짜 = f"2025-{month}-{day}"
    기준날짜 = pd.to_datetime(기준날짜).strftime('%Y-%m-%d')  # 날짜 포맷 보정
    df = df[df['취소(포기)\n일자'] == 기준날짜]

    # 각 신청단계별 개수 계산
    승인불가_개수 = df[df['신청단계'] == '승인불가'].shape[0]
    신청포기_개수 = df[df['신청단계'] == '신청포기'].shape[0]
    지원취소_개수 = df[df['신청단계'] == '지원취소'].shape[0]
    
    # 전체 개수 (세 개의 합계)
    전체_개수 = 승인불가_개수 + 신청포기_개수 + 지원취소_개수
    
    기준날짜_취소_개수_리스트.append({
        '날짜': 기준날짜, 
        '개수': 전체_개수,
        '승인불가': 승인불가_개수,
        '신청포기': 신청포기_개수,
        '지원취소': 지원취소_개수
    })

new_df = pd.DataFrame(기준날짜_취소_개수_리스트)
new_df.to_excel('기준날짜_취소_개수_집계.xlsx', index=False)

In [54]:
df = pd.read_excel('Greet_Subsidy_2Q.xlsx', sheet_name='지급신청', header=3)

# 배분일별 개수 집계 및 데이터프레임화
배분일_개수 = df['배분일'].value_counts().reset_index()
배분일_개수.columns = ['날짜', '개수']

# 날짜 오름차순 정렬
배분일_개수 = 배분일_개수.sort_values('날짜').reset_index(drop=True)

# 엑셀로 저장
배분일_개수.to_excel('배분일별_개수_집계.xlsx', index=False)

In [55]:
df = pd.read_excel('Greet_Subsidy_2Q.xlsx', sheet_name='지급신청', header=3)

# 배분일별 개수 집계 및 데이터프레임화
배분일_개수 = df['지급\n신청일'].value_counts().reset_index()
배분일_개수.columns = ['날짜', '개수']

# 날짜 오름차순 정렬
배분일_개수 = 배분일_개수.sort_values('날짜').reset_index(drop=True)

# 엑셀로 저장
배분일_개수.to_excel('지급일별_개수_집계.xlsx', index=False)

In [56]:
df = pd.read_excel('Greet_Subsidy_2Q.xlsx', sheet_name='지급신청', header=3)
df_ev = pd.read_excel('EV_Q2.xlsx')

배분일_rn = df[['배분일', 'RN']]
지급신청일_rn = df_ev[['지급신청일자', '제조수입사\n관리번호']]

# 날짜 컬럼을 datetime으로 변환 후 날짜만 추출
배분일_rn['배분일'] = pd.to_datetime(배분일_rn['배분일'], errors='coerce').dt.strftime('%Y-%m-%d')
지급신청일_rn['지급신청일자'] = pd.to_datetime(지급신청일_rn['지급신청일자'], errors='coerce').dt.strftime('%Y-%m-%d')

# 'RN'과 '제조수입사\n관리번호'를 기준으로 left join
merged_df = 배분일_rn.merge(
    지급신청일_rn, 
    left_on='RN', 
    right_on='제조수입사\n관리번호', 
    how='left'
)

# 결과 컬럼만 남기기
result_df = merged_df[['RN', '배분일', '지급신청일자']]

result_df.to_excel('지급_집계.xlsx', index=False)


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_7948\3672846104.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  배분일_rn['배분일'] = pd.to_datetime(배분일_rn['배분일'], errors='coerce').dt.strftime('%Y-%m-%d')
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_7948\3672846104.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  지급신청일_rn['지급신청일자'] = pd.to_datetime(지급신청일_rn['지급신청일자'], errors='coerce').dt.strftime('%Y-%m-%d')


In [57]:
df = pd.read_excel('지급_집계.xlsx')

# 배분일과 지급신청일자가 다른(미신청) 데이터만 추출
미신청_df = df[df['배분일'] != df['지급신청일자']]

# '배분일'별 미신청 개수 집계
미신청_집계 = 미신청_df['배분일'].value_counts().reset_index()
미신청_집계.columns = ['날짜', '미신청']

미신청_집계 = 미신청_집계.sort_values('날짜').reset_index(drop=True)
미신청_집계.to_excel('미신청_집계.xlsx', index=False)